In [206]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from matplotlib import style
import datetime
import seaborn as sns
import numpy as np
from functools import reduce

## This Notebook combines all the county data 

In [207]:
###PA.gov
def yn_switch(opdfnew):
    opdfnew["Narcan Admin"] = opdfnew['Naloxone Administered'].map(lambda x: 1 if x == "Y" else 0)
    opdfnew["Survive"] = opdfnew['Survive'].map(lambda x: 1 if x == "Y" else 0)

    return(opdfnew)   
df = pd.read_csv('../data/PAGOV.csv')
df.head()
opiod_list = ['CARFENTANIL','FENTANYL','FENTANYL ANALOG/OTHER SYNTHETIC OPIOID','HEROIN','METHADONE','PHARMACEUTICAL OPIOID','SUBOXONE']

# Reduce incidents to ones with specific opioid
opdf = df.loc[df['Susp OD Drug Desc'].isin(opiod_list)]

# Store relevant information, duplicates
opdf = opdf[['Incident ID','Incident County Name','Incident Date','Victim ID','Gender Desc','Day','Naloxone Administered',
             'Age Range','Survive','Response Desc','Incident Time']].drop_duplicates()


opdf['Incident Date ym'] = pd.to_datetime(opdf['Incident Date']).dt.to_period('Y')

opdf['Incident Date ym'] = opdf['Incident Date ym'].astype(str)
opdf['Incident Date ym'] = opdf['Incident Date ym'].astype(int)

oppdf = opdf.groupby(['Incident Date ym'])[['Incident ID']].count().reset_index()
opdfnew = yn_switch(opdf.copy())

#limitign
opdfnew = opdfnew[['Incident ID','Incident County Name',
                   'Narcan Admin','Survive','Incident Date ym']]

#Renaming for joins later
opdfnew  = opdfnew.rename(columns={'Incident County Name': 'County','Incident Date ym':'Year'})

##Getting totals

opdfnew['Total Overdoses Per County'] = opdfnew.groupby(['County','Year'])['County'].transform('count')

opdfnew['Percent Narcan Admin Per County/Year'] = opdfnew.groupby(['County','Year'])['Narcan Admin'].transform('mean')
opdfnew['Percent Survive Overdose Per County/Year'] = opdfnew.groupby(['County','Year'])['Survive'].transform('mean')
opdfnew['Total Number of PA.gov Overdoses Per County/Year'] = opdfnew.groupby(['County','Year'])['Incident ID'].transform('count')
##Limiting

opdfnew = opdfnew[['County','Year','Percent Narcan Admin Per County/Year',
                   'Percent Survive Overdose Per County/Year',
                   'Total Number of PA.gov Overdoses Per County/Year']].drop_duplicates()



In [208]:
opdfnew.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total Number of PA.gov Overdoses Per County/Year
2,Delaware,2018,0.656250,0.703125,64
4,Chester,2018,0.813397,0.751196,209
5,Beaver,2018,0.685185,0.814815,54
6,Bucks,2018,0.739895,0.804921,569
8,Philadelphia,2018,0.951220,0.926829,41


In [209]:
### Next, Takeback and treaments
takebackdf = pd.read_csv('../data/Prescription_Drug_Take-Back_Box_Locations_County_Drug_and_Alcohol_Programs.csv')
### There is some datacleanup needed as some counties are in all caps and listed as PA

def fix(df):
    for i, row in df.iterrows():
        try:
            cngcnty = row['County'].split(',')[0]
            cngcnty = str.title(cngcnty)
            df.at[i,'County'] = cngcnty
        except:
            pass
    return(df)        
takebackdf1 = fix(takebackdf.copy())

takebackdf1['Total of Take Back Locations by County'] = takebackdf1.groupby('County')['County'].transform('count')




###Loading treatment
treatdf = pd.read_csv('../data/Drug_and_Alcohol_Treatment_Facilities_May_2018_County_Drug_and_Alcohol_Programs.csv')

treatdf['Total of Treatment Locations by County'] = treatdf.groupby('County')['County'].transform('count')

treatdf = treatdf[['County','Total of Treatment Locations by County']]

### Merging dataframes
dfcnt2 = pd.merge(takebackdf1,treatdf, how="inner",left_on = 'County',right_on = 'County')

dftakebackandtreatment = dfcnt2[['County','Total of Take Back Locations by County',
                                 'Total of Treatment Locations by County']].drop_duplicates()



In [210]:
dfmerge1 = pd.merge(opdfnew,dftakebackandtreatment, how="inner",left_on = 'County',right_on = 'County')

In [211]:
dfmerge1.head()  ##PA.gov and takebacks/treatments

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total Number of PA.gov Overdoses Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County
0,Delaware,2018,0.656250,0.703125,64,55,30
1,Delaware,2019,0.717949,0.615385,39,55,30
2,Delaware,2020,0.555556,0.587302,63,55,30
3,Delaware,2021,0.560000,0.700000,50,55,30
4,Delaware,2022,0.500000,0.714286,28,55,30


In [212]:
###Risky Precribing dataset
df = pd.read_csv('../data/Risky_Prescribing_Measures_Quarter_3_2016_-_Current_Quarterly_County___Statewide_Health.csv')

### Removing PA as to not count twice
df = df[df['County'] != 'Pennsylvania']

##Suming per county/year

df['Total Risky Prescibing per County/Year'] = df.groupby(['County','Year'])['Rate or Count'].transform('sum')

#limiting
dfrisky = df[['County','Year','Total Risky Prescibing per County/Year']]




In [213]:
dfmerge1['Year'] = dfmerge1['Year'].astype(int)

In [214]:
dfrisky.head()

,County,Year,Total Risky Prescibing per County/Year
1,Lackawanna,2020,36468.040054
4,Mifflin,2021,3224.363865
5,Schuylkill,2021,11536.172907
6,Butler,2019,26160.090000
7,Snyder,2016,4034.040000


In [215]:
dfmerge1.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total Number of PA.gov Overdoses Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County
0,Delaware,2018,0.656250,0.703125,64,55,30
1,Delaware,2019,0.717949,0.615385,39,55,30
2,Delaware,2020,0.555556,0.587302,63,55,30
3,Delaware,2021,0.560000,0.700000,50,55,30
4,Delaware,2022,0.500000,0.714286,28,55,30


In [216]:
##Merging risky dataset
dfmerge2 = pd.merge(dfmerge1,dfrisky, how="inner",left_on = ['County','Year'],right_on = ['County','Year'])

In [217]:
dfmerge2.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total Number of PA.gov Overdoses Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County,Total Risky Prescibing per County/Year
0,Delaware,2018,0.65625,0.703125,64,55,30,87748.56
1,Delaware,2018,0.65625,0.703125,64,55,30,87748.56
2,Delaware,2018,0.65625,0.703125,64,55,30,87748.56
3,Delaware,2018,0.65625,0.703125,64,55,30,87748.56
4,Delaware,2018,0.65625,0.703125,64,55,30,87748.56


In [218]:
### Dispensation_Data
df = pd.read_csv('../data/Dispensation_Data.csv')
###Keeping age groups, removing all ages ## will have to do Genders seperatly as 
##They don't count them together
df = df[df['Age Group'] == 'All Ages']
df = df[df['Gender'] == 'All Genders']

df  = df.rename(columns={'County Name': 'County'})
df['Total Dispensation per County/Year'] = df.groupby(['County','Year'])['Rate or Count'].transform('sum')
dispdf = df[['County','Year','Total Dispensation per County/Year']].drop_duplicates()


In [219]:
##Merging Dispensation
dfmerge3 = pd.merge(dfmerge2,dispdf, how="inner",left_on = ['County','Year'],right_on = ['County','Year'])


In [220]:
dfmerge3 = dfmerge3.drop_duplicates()

In [221]:
dfmerge3.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total Number of PA.gov Overdoses Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County,Total Risky Prescibing per County/Year,Total Dispensation per County/Year
0,Delaware,2018,0.656250,0.703125,64,55,30,87748.560000,653144.920000
64,Delaware,2019,0.717949,0.615385,39,55,30,70744.460000,582654.380000
128,Delaware,2020,0.555556,0.587302,63,55,30,57569.091674,514813.120000
192,Delaware,2021,0.560000,0.700000,50,55,30,38628.769756,367033.862535
240,Chester,2018,0.813397,0.751196,209,26,25,64356.480000,480173.990000


## Arrests

In [222]:
df = pd.read_csv('../data/Opioid_Seizures_and_Arrests_CY_2013_-_Current_Quarterly_County_State_Police(1).csv')


In [223]:
df = df.rename(columns={'County Name': 'County'})

In [224]:
df1Fentanyl = df.assign(Fentanyl_Incident = np.where(df['Drug']=='Fentanyl',df['Incident Count'],0))\
   .groupby(['County','Year']).agg({'Fentanyl_Incident':sum}).reset_index().drop_duplicates()

df1Heroin= df.assign(Heroin_Incident = np.where(df['Drug']=='Heroin',df['Incident Count'],0))\
   .groupby(['County','Year']).agg({'Heroin_Incident':sum}).reset_index().drop_duplicates()

df1Opium= df.assign(Opium_Incident = np.where(df['Drug']=='Opium',df['Incident Count'],0))\
   .groupby(['County','Year']).agg({'Opium_Incident':sum}).reset_index().drop_duplicates()

In [225]:
df2Fentanyl = df.assign(Fentanyl_Arrests = np.where(df['Drug']=='Fentanyl',df['Arrests'],0))\
   .groupby(['County','Year']).agg({'Fentanyl_Arrests':sum}).reset_index().drop_duplicates()

df2Heroin= df.assign(Heroin_Arrests = np.where(df['Drug']=='Heroin',df['Arrests'],0))\
   .groupby(['County','Year']).agg({'Heroin_Arrests':sum}).reset_index().drop_duplicates()

df2Opium= df.assign(Opium_Arrests = np.where(df['Drug']=='Opium',df['Arrests'],0))\
   .groupby(['County','Year']).agg({'Opium_Arrests':sum}).reset_index().drop_duplicates()

In [226]:
df3Fentanyl = df.assign(Fentanyl_Quantity = np.where(df['Drug']=='Fentanyl',df['Drug Quantity'],0))\
   .groupby(['County','Year']).agg({'Fentanyl_Quantity':sum}).reset_index().drop_duplicates()

df3Heroin= df.assign(Heroin_Quantity = np.where(df['Drug']=='Heroin',df['Drug Quantity'],0))\
   .groupby(['County','Year']).agg({'Heroin_Quantity':sum}).reset_index().drop_duplicates()

df3Opium= df.assign(Opium_Quantity = np.where(df['Drug']=='Opium',df['Drug Quantity'],0))\
   .groupby(['County','Year']).agg({'Opium_Quantity':sum}).reset_index().drop_duplicates()

In [227]:
data_frames = [df1Fentanyl,df1Heroin,df1Opium,df2Fentanyl,df2Heroin,df2Opium,df3Fentanyl,
              df3Heroin,df3Opium]  ## combining dataframes

## Merging dataframe
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['County','Year'],
                                            how='outer'), data_frames)


##Creating computed collumns

df_merged['%Incidents Fentanyl'] = (df_merged['Fentanyl_Incident']/ 
(df_merged['Fentanyl_Incident'] +df_merged['Opium_Incident'] + df_merged['Heroin_Incident']))

df_merged['%Arrests Fentanyl'] = (df_merged['Fentanyl_Arrests']/ 
(df_merged['Fentanyl_Arrests'] + df_merged['Opium_Arrests'] + df_merged['Heroin_Arrests']))

df_merged['%Quantity Fentanyl'] = (df_merged['Fentanyl_Quantity']/ 
(df_merged['Fentanyl_Quantity'] + df_merged['Opium_Quantity'] + df_merged['Heroin_Quantity']))
dfarrestsfinal = df_merged

In [228]:
dfmerge4 = pd.merge(dfmerge3,dfarrestsfinal, how="inner",left_on = ['County','Year'],right_on = ['County','Year'])



In [229]:
dfmerge4 =dfmerge4.drop_duplicates()

In [230]:
dfmerge4.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total Number of PA.gov Overdoses Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County,Total Risky Prescibing per County/Year,Total Dispensation per County/Year,Fentanyl_Incident,...,Opium_Incident,Fentanyl_Arrests,Heroin_Arrests,Opium_Arrests,Fentanyl_Quantity,Heroin_Quantity,Opium_Quantity,%Incidents Fentanyl,%Arrests Fentanyl,%Quantity Fentanyl
0,Delaware,2018,0.656250,0.703125,64,55,30,87748.560000,653144.920000,5,...,0,3,164,0,0.02077,0.538028,0.000000,0.033557,0.017964,0.037169
1,Delaware,2019,0.717949,0.615385,39,55,30,70744.460000,582654.380000,15,...,0,19,198,0,5.27221,5.501150,0.000000,0.079787,0.087558,0.489375
2,Delaware,2020,0.555556,0.587302,63,55,30,57569.091674,514813.120000,23,...,0,27,202,0,0.16478,1.270373,0.000000,0.126374,0.117904,0.114817
3,Delaware,2021,0.560000,0.700000,50,55,30,38628.769756,367033.862535,27,...,2,39,150,2,17.05824,4.397683,0.000012,0.178808,0.204188,0.795036
4,Chester,2018,0.813397,0.751196,209,26,25,64356.480000,480173.990000,5,...,2,6,64,2,1.09655,0.106369,0.000250,0.060241,0.083333,0.911385


## County population

In [231]:
df = pd.read_csv('../data/County_pop.csv')



In [232]:
df.head()

,County,2010,2020,Numeric_change,Percent_Change
0,Pennsylvania,12702379,13002700,300321,0.024
1,Adams County,101407,103852,2445,0.024
2,Allegheny County,1223348,1250578,27230,0.022
3,Armstrong County,68941,65558,-3383,-0.049
4,Beaver County,170539,168215,-2324,-0.014


In [233]:
df = df.rename(columns={'Numeric_change': 'County Numeric Change Since 2010',
                         'Percent_Change': 'County Percent Change Since 2010',
                         '2020':'County Population'})

In [234]:
df = df[['County','County Population','County Numeric Change Since 2010','County Percent Change Since 2010']]

In [235]:
df.head()

,County,County Population,County Numeric Change Since 2010,County Percent Change Since 2010
0,Pennsylvania,13002700,300321,0.024
1,Adams County,103852,2445,0.024
2,Allegheny County,1250578,27230,0.022
3,Armstrong County,65558,-3383,-0.049
4,Beaver County,168215,-2324,-0.014


In [236]:
df = df[df['County'] != 'Pennsylvania']


In [237]:
def fixcnty(df):
    for i, row in df.iterrows():
        cngcnty = row['County'].split('County')[0]
        cngcnty = str.title(cngcnty).strip()
        df.at[i,'County'] = cngcnty
    return(df)       

In [238]:
dfcntpop = fixcnty(df.copy())

In [239]:
df1.head()

,County,County Population,County Numeric Change Since 2010,County Percent Change Since 2010
1,Adams,103852,2445,0.024
2,Allegheny,1250578,27230,0.022
3,Armstrong,65558,-3383,-0.049
4,Beaver,168215,-2324,-0.014
5,Bedford,47577,-2185,-0.044


In [240]:
dfcntpop 

,County,County Population,County Numeric Change Since 2010,County Percent Change Since 2010
1,Adams,103852,2445,0.024
2,Allegheny,1250578,27230,0.022
3,Armstrong,65558,-3383,-0.049
4,Beaver,168215,-2324,-0.014
5,Bedford,47577,-2185,-0.044
...,...,...,...,...
63,Washington,209349,1529,0.007
64,Wayne,51155,-1667,-0.032
65,Westmoreland,354663,-10506,-0.029
66,Wyoming,26069,-2207,-0.078


In [241]:
dfmerge4.to_csv('Mastercounty.csv')

In [242]:
dfmerge5 = pd.merge(dfmerge4,dfcntpop, how="inner",left_on = 'County',right_on = 'County')



In [243]:
dfmerge5.columns

Index(['County', 'Year', 'Percent Narcan Admin Per County/Year',
       'Percent Survive Overdose Per County/Year',
       'Total Number of PA.gov Overdoses Per County/Year',
       'Total of Take Back Locations by County',
       'Total of Treatment Locations by County',
       'Total Risky Prescibing per County/Year',
       'Total Dispensation per County/Year', 'Fentanyl_Incident',
       'Heroin_Incident', 'Opium_Incident', 'Fentanyl_Arrests',
       'Heroin_Arrests', 'Opium_Arrests', 'Fentanyl_Quantity',
       'Heroin_Quantity', 'Opium_Quantity', '%Incidents Fentanyl',
       '%Arrests Fentanyl', '%Quantity Fentanyl', 'County Population',
       'County Numeric Change Since 2010', 'County Percent Change Since 2010'],
      dtype='object')

In [244]:
dfmerge5['Legal Opiod Dispense per Person'] = (dfmerge5['Total Dispensation per County/Year']) / (dfmerge5['County Population'])
dfmerge5['Risky Opiod Prescribe per Person'] = (dfmerge5['Total Risky Prescibing per County/Year']) / (dfmerge5['County Population'])
dfmerge5['Num of PA.gov Overdose per Person'] = (dfmerge5['Total Number of PA.gov Overdoses Per County/Year']) / (dfmerge5['County Population'])




In [245]:
###Trying for Tablau
dfmerge6 = dfmerge5[['County','Year','Total Number of PA.gov Overdoses Per County/Year','Year', 'Percent Narcan Admin Per County/Year',
                    'Percent Survive Overdose Per County/Year','%Arrests Fentanyl',
                     'Legal Opiod Dispense per Person',
                     'Risky Opiod Prescribe per Person',
                    ]]

In [246]:
dfmerge6.head(20)

,County,Year,Total Number of PA.gov Overdoses Per County/Year,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,%Arrests Fentanyl,Legal Opiod Dispense per Person,Risky Opiod Prescribe per Person,Num of PA.gov Overdose per Person
0,Delaware,2018,64,2018,0.656250,0.703125,0.017964,1.132301,0.152122,0.000111
1,Delaware,2019,39,2019,0.717949,0.615385,0.087558,1.010097,0.122644,0.000068
2,Delaware,2020,63,2020,0.555556,0.587302,0.117904,0.892487,0.099803,0.000109
3,Delaware,2021,50,2021,0.560000,0.700000,0.204188,0.636295,0.066967,0.000087
4,Chester,2018,209,2018,0.813397,0.751196,0.083333,0.898507,0.120425,0.000391
5,Chester,2019,175,2019,0.725714,0.805714,0.047619,0.805494,0.098048,0.000327
6,Chester,2020,148,2020,0.790541,0.844595,0.198473,0.728157,0.083379,0.000277
7,Chester,2021,75,2021,0.493333,0.653333,0.217822,0.535939,0.058861,0.000140
8,Beaver,2018,54,2018,0.685185,0.814815,0.000000,1.613420,0.186311,0.000321
9,Beaver,2019,43,2019,0.651163,0.674419,0.000000,1.399609,0.139008,0.000256


In [248]:
dfmerge6.to_csv('dfmerge6.csv')